# Regression Analysis of Driver Safety

<div class="alert alert-block alert-success"> The following <b>regression analysis</b> is used to determine the predictive power of Fleet Safety driver data. The data includes every field employee driver who was or was not involved in a traffic incident from 2016-2020. The two datasets used in this analysis are: <b>Driver Accidents</b> and <b>Driver Trainings</b>. The <b>purpose</b> of this analysis is to determine whether the <b>current data</b> we have is enough to move forward with a prescriptive ML model <b>OR</b> if we need to capture <b>more</b> attributes.  </div>

## Packages

<div class="alert alert-block alert-success"> <b>Packages</b> that <b>may</b> need to be installed </div>

In [1]:
# !python3 -mpip install statsmodels
# !python3 -mpip install researchpy
# !python3 -mpip install sklearn
# !python3 -mpip install xgboost

## Optional

<div class="alert alert-block alert-success"> Allows <b>multiple outputs</b> in one cell </div>

In [2]:
# to have multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Modules

<div class="alert alert-block alert-success"> Modules to import / load  </div>

In [ ]:
import re
import xlrd
import openpyxl
from openpyxl import Workbook
import xlsxwriter

import pandas as pd
from pandas import read_csv

import numpy as np
from numpy import mean, std 

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.formula.api import ols

import researchpy as rp

import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import xgboost
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

## Original Datasets

<div class="alert alert-block alert-success"> Reading in original <b>ACCIDENTS</b> and <b>TRAINING</b> datasets  </div>

In [25]:
accidents = pd.read_excel('accident_file.xlsx', engine='openpyxl')
training = pd.read_excel('training_file.xlsx', engine='openpyxl')

accidents.shape
accidents.head()

training.shape
training.head()

# Data Conditioning

## Renaming employee id columns

<div class="alert alert-block alert-success"> Renaming <b>employee id</b> columns in BOTH datasets to have <b>same name</b> so we can left join on training dataset</div>

In [5]:
# renaming id columns to same name to join dfs
accidents.rename(columns={'Employee Number': 'employee_id'}, inplace=True)
training.rename(columns={'Employee ID': 'employee_id'}, inplace=True)

## Left joining dataframes

<div class="alert alert-block alert-success"> <b>LEFT joining</b> training and accidents dataframes onto training dataframe and making minor cleaning/shaping </div>

In [26]:
#left join on training dataset
df = pd.merge(training, accidents, on="employee_id", how="left")

#converting all columns to lowercase and replacing spaces with underscore for easier use
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

#renaming column to provider for easier use
df.rename(columns={'dc_5.0.education.education_provider': 'provider'}, inplace=True)

#dropping duplicate records to ensure accurate results in later analysis
df = df.drop_duplicates(subset=['employee_id', 'assigned_on_date'])

df.shape
df.head()

In [7]:
accident_BEFORE_training = df
accident_AFTER_training = df

## Filtering to ONLY incidents BEFORE trainings assigned

<div class="alert alert-block alert-success"> Shaping <b>BEFORE training</b> dataframe</div>

In [27]:
#ensuring that all date columns are referenced as datetime types
accident_BEFORE_training['assigned_on_date'] = pd.to_datetime(accident_BEFORE_training['assigned_on_date'])
accident_BEFORE_training['loss_date'] = pd.to_datetime(accident_BEFORE_training['loss_date'])

#filtering columns and rows that got into incident BEFORE training was assigned
accident_BEFORE_training = accident_BEFORE_training.loc[accident_BEFORE_training.assigned_on_date <= accident_BEFORE_training.loss_date]
accident_BEFORE_training.shape
accident_BEFORE_training.head()
list(accident_BEFORE_training.columns)
# accident_BEFORE_training.to_excel('accident_BEFORE_training1234.xlsx')

## One-hot-encode - BEFORE TRAINING DF

<div class="alert alert-block alert-success"> One-hot-encoding <b>BEFORE training</b> dataframe with ONLY TRAINING modules and training PROVIDERS to compare with "FULL" and "AFTER training" datasets. This is to be able to use in ML model later since it is all <b>categorical data</b> </div>

In [28]:
# one-hot-encode BEFORE_training df
one_hot_encode = pd.crosstab(accident_BEFORE_training.employee_id, accident_BEFORE_training.education).gt(0).astype(int)
provider = accident_BEFORE_training[['employee_id']].join(pd.get_dummies(accident_BEFORE_training['provider'])).groupby('employee_id').max()
BEFORE_training = pd.concat([one_hot_encode, provider], axis=1)
BEFORE_training['Total_Incidents'] = accident_BEFORE_training.groupby('employee_id')['claim_id'].nunique()
BEFORE_training.columns = BEFORE_training.columns.str.lower()
BEFORE_training.columns = BEFORE_training.columns.str.replace(' ','_')
BEFORE_training.shape
BEFORE_training.head()

<div class="alert alert-block alert-success"> One-hot-encoding "AFTER training" dataframe with only TRAINING modules and training PROVIDERS to compare with "FULL" and "BEFORE training" datasets. This is to be able to use in ML model later since it is all categorical data </div>

In [29]:
one_hot_encode = pd.crosstab(df.employee_id, df.education).gt(0).astype(int)
provider = df[['employee_id']].join(pd.get_dummies(df['provider'])).groupby('employee_id').max()
onehot_df = pd.concat([one_hot_encode, provider], axis=1)

#adding a column to record total number of incidents each employee has encountered
onehot_df['Total_Incidents'] = df.groupby('employee_id')['claim_id'].nunique()

onehot_df.columns = onehot_df.columns.str.lower()
onehot_df.columns = onehot_df.columns.str.replace(' ','_')
onehot_df.shape
onehot_df.head()

# Regression Analysis

## Regression Analysis - COMBINED dataset (onehot_df) 

<div class="alert alert-block alert-success"> Creating <b>X and Y variables</b> for regression analysis on <b>full dataframe </b>to identify statistical significance such as <b>R-squared value</b>, and determine predictive power  </div>

In [15]:
#selecting X variable
X = onehot_df.iloc[:, :-1]
X.shape

#selecting Y variable
Y = onehot_df.iloc[:, -1][:, None]
Y.shape
X_test = onehot_df

(3195, 34)

(3195, 1)

<div class="alert alert-block alert-success"> Creating <b>training</b> and <b>testing</b> datasets - test size is <b>20%</b>  </div>

In [16]:
#creating training and testing datasets with test size at 20%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2556, 34)
(639, 34)
(2556, 1)
(639, 1)


<div class="alert alert-block alert-success"> <b>OLS Regression</b> results for FULL dataframe (onehot_df)  </div>

In [17]:
X_train_Sm= sm.add_constant(X_train)
X_train_Sm= sm.add_constant(X_train)
ls=sm.OLS(y_train,X_train_Sm).fit()
print(ls.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     33.26
Date:                Mon, 26 Jul 2021   Prob (F-statistic):          9.40e-163
Time:                        19:27:04   Log-Likelihood:                -1414.8
No. Observations:                2556   AIC:                             2894.
Df Residuals:                    2524   BIC:                             3081.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

<div class="alert alert-block alert-success"> <b>R-squared value</b> of <b>0.290</b> is <b>too small</b> (R-squared value indicates strong or weak predictive power). 0.290 basically means that only <b>29%</b> of the total number of incidents can be explained from the data - <b>this is not enough</b>.  <br>

This reveals that <b>more holistic driving profiles</b> are needed to perform prescriptive analytics, such as the development of <b>full driver profiles</b>, weather conditions on days that employees drove and an accident <b>NOT</b> happen, etc. </br> </div>

## Summary

<div class="alert alert-block alert-success"> A <b>regression analysis</b> was performed on the <b>driver data</b> to determine its statistical significance/predictive power to use in a prescriptive ML model. The data includes detailed information about every employee driver who was or was not involved in a traffic incident from 2016-2020. The two datasets used in this report are: <b>Driver Incidents</b> and <b>Driver Trainings</b>. <br>
    
Both datasets were LEFT joined onto the Driver Trainings dataset to match drivers with their appropriate trainings completed and to eliminate duplicate values. A regression analysis was performed on this compiled dataset to uncover the explainability of the data. </br> <br> 

An <b>R-squared value</b> of <b>0.290</b> was identified which is <b>very small</b> (R-squared value indicates strong or weak predictive power). 0.290  means that only <b>29%</b> of the total number of incidents can be explained from the data - <b>this is not high enough to rely upon in a predictive model</b>.</br> <br>

<b>External factors</b> that are not in the current data play a more crucial role in explaining the causes of the accidents. A more holistic driver profile is needed, such as the road and weather conditions on days a driver does <b>NOT</b> get into an accident. These factors must be captured in order to build a robust predictive model in the future</br> <br>

</br><br>

It would be helpful if drivers could record their specific routes and the road and weather conditions on EACH day they drive</br>
</div>

### BEFORE_training correlations

In [22]:
model = XGBClassifier()
model.fit(X_train, y_train)

[19:28:34] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=64, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [24]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100))

Accuracy: 73.40%
